In [1]:
import collections
import glob
import itertools
import json
import operator
import os
import re

import natsort
import matplotlib.pyplot as plt
import pandas as pd

%matplotlib inline
plt.rcParams['figure.figsize'] = [20, 20]

from seq2struct.utils import evaluation

In [2]:
os.chdir('..')

In [3]:
accuracy_per_run = collections.defaultdict(dict)
all_metrics = []
metric_types = set()
rows = []
for d in sorted(glob.glob('logdirs/20190201-hs-allmatches-anysplit-multimean/*')):
    exp_name = os.path.basename(d)
    exp_vars = re.match('filt-([^_]+)_st-([^_]+)_nt-([^_]+)', exp_name).groups()

    infer_paths = glob.glob(os.path.join(d, 'infer-val-20191031-step*-bs1.jsonl'))
    
    for infer_path in infer_paths:
        inferred = [json.loads(line) for line in open(infer_path)]
        
        step = int(re.search('step(\d+)', infer_path).group(1))
        _, metrics = evaluation.compute_metrics(
            'configs/hearthstone/nl2code.jsonnet', '', 'val', infer_path)
        all_metrics.append((exp_name, step, metrics))
        metric_types.update(metrics.keys())
    print(d)
    
metric_types = tuple(sorted(metric_types))
flat_df = pd.DataFrame(
    [(exp_name, step) + tuple(metrics.get(t) for t in metric_types) for exp_name, step, metrics in all_metrics],
    columns=('exp_name', 'step') + metric_types)

logdirs/20190201-hs-allmatches-anysplit-multimean/filt-contains-hole_st-cov-examples_nt-10
logdirs/20190201-hs-allmatches-anysplit-multimean/filt-contains-hole_st-cov-examples_nt-20
logdirs/20190201-hs-allmatches-anysplit-multimean/filt-contains-hole_st-cov-examples_nt-40
logdirs/20190201-hs-allmatches-anysplit-multimean/filt-contains-hole_st-cov-examples_nt-80
logdirs/20190201-hs-allmatches-anysplit-multimean/filt-contains-hole_st-cov-xent_nt-10
logdirs/20190201-hs-allmatches-anysplit-multimean/filt-contains-hole_st-cov-xent_nt-20
logdirs/20190201-hs-allmatches-anysplit-multimean/filt-contains-hole_st-cov-xent_nt-40
logdirs/20190201-hs-allmatches-anysplit-multimean/filt-contains-hole_st-cov-xent_nt-80
logdirs/20190201-hs-allmatches-anysplit-multimean/filt-none_st-cov-examples_nt-10
logdirs/20190201-hs-allmatches-anysplit-multimean/filt-none_st-cov-examples_nt-20
logdirs/20190201-hs-allmatches-anysplit-multimean/filt-none_st-cov-examples_nt-40
logdirs/20190201-hs-allmatches-anysplit-mu

In [7]:
flat_df.loc[flat_df.groupby('exp_name')['exact match'].idxmax()]

,exp_name,step,corpus BLEU,exact match,sentence BLEU
17,filt-contains-hole_st-cov-examples_nt-10,1400,0.762570,0.151515,0.776417
27,filt-contains-hole_st-cov-examples_nt-20,2400,0.706141,0.090909,0.704459
56,filt-contains-hole_st-cov-examples_nt-40,2500,0.732747,0.166667,0.758392
80,filt-contains-hole_st-cov-examples_nt-80,2200,0.756422,0.196970,0.776508
118,filt-contains-hole_st-cov-xent_nt-10,2100,0.768508,0.151515,0.780367
143,filt-contains-hole_st-cov-xent_nt-20,2100,0.767917,0.166667,0.776129
168,filt-contains-hole_st-cov-xent_nt-40,2100,0.769610,0.181818,0.768441
177,filt-contains-hole_st-cov-xent_nt-80,2400,0.762490,0.151515,0.750810
208,filt-none_st-cov-examples_nt-10,2300,0.732471,0.151515,0.740296
227,filt-none_st-cov-examples_nt-20,2400,0.771503,0.181818,0.783085


In [8]:
flat_df.loc[flat_df.groupby('exp_name')['corpus BLEU'].idxmax()]

,exp_name,step,corpus BLEU,exact match,sentence BLEU
6,filt-contains-hole_st-cov-examples_nt-10,2500,0.785207,0.106061,0.780821
34,filt-contains-hole_st-cov-examples_nt-20,2000,0.746716,0.075758,0.736189
59,filt-contains-hole_st-cov-examples_nt-40,2000,0.764626,0.106061,0.769131
81,filt-contains-hole_st-cov-examples_nt-80,2500,0.774179,0.166667,0.780344
108,filt-contains-hole_st-cov-xent_nt-10,2300,0.783270,0.121212,0.764812
130,filt-contains-hole_st-cov-xent_nt-20,2200,0.782329,0.136364,0.778930
168,filt-contains-hole_st-cov-xent_nt-40,2100,0.769610,0.181818,0.768441
193,filt-contains-hole_st-cov-xent_nt-80,2100,0.764970,0.136364,0.751688
205,filt-none_st-cov-examples_nt-10,2200,0.762103,0.090909,0.743000
231,filt-none_st-cov-examples_nt-20,2500,0.782529,0.151515,0.762898


# Corpus BLEU

In [11]:
flat_df[flat_df['corpus BLEU'] == flat_df['corpus BLEU'].max()]

,exp_name,step,corpus BLEU,exact match,sentence BLEU
308,filt-none_st-cov-xent_nt-10,2300,0.785476,0.151515,0.773789


In [12]:
# Beam size 1
!CUDA_VISIBLE_DEVICES= python infer.py --config configs/hearthstone-idioms/nl2code-0201-allmatches-anysplit-multimean.jsonnet --logdir logdirs/20190201-hs-allmatches-anysplit-multimean/filt-none_st-cov-xent_nt-10 --config-args "{filt: 'none', st: 'cov-xent', nt: 10}" --output __LOGDIR__/infer-test-20191031-step02300-bs1.jsonl --step 2300 --section test --beam-size 1

WARNING <class 'seq2struct.models.enc_dec.EncDecModel.Preproc'>: superfluous {'name': 'EncDec'}
WARNING <class 'seq2struct.models.enc_dec.EncDecModel'>: superfluous {'encoder_preproc': {'max_count': 5000, 'save_path': 'data/hearthstone-idioms-20190201/all-matches-trees-anysplit/filt-none_st-cov-xent_nt-10/nl2code/', 'min_freq': 3}, 'decoder_preproc': {'max_count': 5000, 'save_path': 'data/hearthstone-idioms-20190201/all-matches-trees-anysplit/filt-none_st-cov-xent_nt-10/nl2code/', 'min_freq': 3, 'use_seq_elem_rules': True, 'grammar': {'name': 'idiom_ast', 'template_file': 'data/hearthstone-idioms-20190201/all-matches-trees-anysplit/filt-none_st-cov-xent_nt-10/templates.json', 'all_sections_rewritten': True, 'base_grammar': {'name': 'python'}}}}
Loading model from logdirs/20190201-hs-allmatches-anysplit-multimean/filt-none_st-cov-xent_nt-10/model_checkpoint-00002300
100%|███████████████████████████████████████████| 66/66 [00:39<00:00,  1.67it/s]


In [13]:
_, metrics = evaluation.compute_metrics(
    'configs/hearthstone/nl2code.jsonnet', '', 'test', 
    'logdirs/20190201-hs-allmatches-anysplit-multimean/filt-none_st-cov-xent_nt-10/infer-test-20191031-step02300-bs1.jsonl')
metrics

OrderedDict([('exact match', 0.10606060606060606),
             ('sentence BLEU', 0.7434579505376531),
             ('corpus BLEU', 0.7480294254902261)])

# Sentence BLEU

In [14]:
flat_df[flat_df['sentence BLEU'] == flat_df['sentence BLEU'].max()]

,exp_name,step,corpus BLEU,exact match,sentence BLEU
230,filt-none_st-cov-examples_nt-20,2200,0.780836,0.166667,0.787985


In [15]:
# Beam size 1
!CUDA_VISIBLE_DEVICES= python infer.py --config configs/hearthstone-idioms/nl2code-0201-allmatches-anysplit-multimean.jsonnet --logdir logdirs/20190201-hs-allmatches-anysplit-multimean/filt-none_st-cov-examples_nt-20 --config-args "{filt: 'none', st: 'cov-examples', nt: 20}" --output __LOGDIR__/infer-test-20191031-step02200-bs1.jsonl --step 2200 --section test --beam-size 1

WARNING <class 'seq2struct.models.enc_dec.EncDecModel.Preproc'>: superfluous {'name': 'EncDec'}
WARNING <class 'seq2struct.models.enc_dec.EncDecModel'>: superfluous {'decoder_preproc': {'grammar': {'base_grammar': {'name': 'python'}, 'all_sections_rewritten': True, 'name': 'idiom_ast', 'template_file': 'data/hearthstone-idioms-20190201/all-matches-trees-anysplit/filt-none_st-cov-examples_nt-20/templates.json'}, 'max_count': 5000, 'min_freq': 3, 'save_path': 'data/hearthstone-idioms-20190201/all-matches-trees-anysplit/filt-none_st-cov-examples_nt-20/nl2code/', 'use_seq_elem_rules': True}, 'encoder_preproc': {'min_freq': 3, 'max_count': 5000, 'save_path': 'data/hearthstone-idioms-20190201/all-matches-trees-anysplit/filt-none_st-cov-examples_nt-20/nl2code/'}}
Loading model from logdirs/20190201-hs-allmatches-anysplit-multimean/filt-none_st-cov-examples_nt-20/model_checkpoint-00002200
100%|███████████████████████████████████████████| 66/66 [00:38<00:00,  1.72it/s]


In [16]:
_, metrics = evaluation.compute_metrics(
    'configs/hearthstone/nl2code.jsonnet', '', 'test', 
    'logdirs/20190201-hs-allmatches-anysplit-multimean/filt-none_st-cov-examples_nt-20/infer-test-20191031-step02200-bs1.jsonl')
metrics

OrderedDict([('exact match', 0.16666666666666666),
             ('sentence BLEU', 0.7704923228057051),
             ('corpus BLEU', 0.7499172018537875)])

# Exact match

In [17]:
flat_df[flat_df['exact match'] == flat_df['exact match'].max()]

,exp_name,step,corpus BLEU,exact match,sentence BLEU
80,filt-contains-hole_st-cov-examples_nt-80,2200,0.756422,0.19697,0.776508


In [18]:
# Beam size 1
!CUDA_VISIBLE_DEVICES= python infer.py --config configs/hearthstone-idioms/nl2code-0201-allmatches-anysplit-multimean.jsonnet --logdir logdirs/20190201-hs-allmatches-anysplit-multimean/filt-contains-hole_st-cov-examples_nt-80 --config-args "{filt: 'contains-hole', st: 'cov-examples', nt: 80}" --output __LOGDIR__/infer-test-20191031-step02200-bs1.jsonl --step 2200 --section test --beam-size 1

WARNING <class 'seq2struct.models.enc_dec.EncDecModel.Preproc'>: superfluous {'name': 'EncDec'}
WARNING <class 'seq2struct.models.enc_dec.EncDecModel'>: superfluous {'decoder_preproc': {'save_path': 'data/hearthstone-idioms-20190201/all-matches-trees-anysplit/filt-contains-hole_st-cov-examples_nt-80/nl2code/', 'min_freq': 3, 'grammar': {'template_file': 'data/hearthstone-idioms-20190201/all-matches-trees-anysplit/filt-contains-hole_st-cov-examples_nt-80/templates.json', 'all_sections_rewritten': True, 'base_grammar': {'name': 'python'}, 'name': 'idiom_ast'}, 'max_count': 5000, 'use_seq_elem_rules': True}, 'encoder_preproc': {'save_path': 'data/hearthstone-idioms-20190201/all-matches-trees-anysplit/filt-contains-hole_st-cov-examples_nt-80/nl2code/', 'min_freq': 3, 'max_count': 5000}}
Loading model from logdirs/20190201-hs-allmatches-anysplit-multimean/filt-contains-hole_st-cov-examples_nt-80/model_checkpoint-00002200
100%|███████████████████████████████████████████| 66/66 [00:40<00:00, 

In [19]:
_, metrics = evaluation.compute_metrics(
    'configs/hearthstone/nl2code.jsonnet', '', 'test', 
    'logdirs/20190201-hs-allmatches-anysplit-multimean/filt-contains-hole_st-cov-examples_nt-80/infer-test-20191031-step02200-bs1.jsonl')
metrics

OrderedDict([('exact match', 0.18181818181818182),
             ('sentence BLEU', 0.7733655603184775),
             ('corpus BLEU', 0.7645024526553997)])

# Compare against baseline

In [33]:
accuracy_per_run = collections.defaultdict(dict)
all_metrics = []
metric_types = set()
rows = []
for d in sorted(glob.glob('logdirs/20181231-nl2code-hearthstone-fef2c5b//*')):
    exp_name = os.path.basename(d)
    exp_vars = re.match('att([^_]+)', exp_name).groups()

    infer_paths = glob.glob(os.path.join(d, 'infer-val-step*-bs1.jsonl'))
    all_scores = []
    for infer_path in infer_paths:
        step = int(re.search('step(\d+)', infer_path).group(1))
        _, metrics = evaluation.compute_metrics(
            'configs/hearthstone/nl2code.jsonnet', '', 'val', infer_path)
        all_scores.append((step, metrics['exact match']))
        all_metrics.append((exp_name, step, metrics))
        metric_types.update(metrics.keys())
    
    all_scores.sort(key=operator.itemgetter(0))
    sorted_scores = sorted(all_scores, reverse=True, key=operator.itemgetter(1))
    rows.append(exp_vars + (len(all_scores),) + (sorted_scores[0] if sorted_scores else (-1, -1)))
    accuracy_per_run[exp_name] = {
        'x': [s[0] for s in all_scores],
        'all': [s[1] for s in all_scores],
    }
    print(d)
    
metric_types = tuple(sorted(metric_types))
df = pd.DataFrame(rows, columns=('att', 'num steps eval', 'step', 'exact match'))
flat_df = pd.DataFrame(
    [(exp_name, step) + tuple(metrics.get(t) for t in metric_types) for exp_name, step, metrics in all_metrics],
    columns=('exp_name', 'step') + metric_types)

logdirs/20181231-nl2code-hearthstone-fef2c5b/att0
logdirs/20181231-nl2code-hearthstone-fef2c5b/att1


In [38]:
df

,att,num steps eval,step,exact match
0,0,25,1900,0.166667
1,1,25,2300,0.196970


## Corpus BLEU

In [35]:
flat_df.loc[flat_df['corpus BLEU'].idxmax()]

exp_name             att1
step                 1900
corpus BLEU      0.789029
exact match      0.151515
sentence BLEU    0.780316
Name: 44, dtype: object

In [36]:
!CUDA_VISIBLE_DEVICES= python infer.py --config configs/hearthstone/nl2code.jsonnet --logdir logdirs/20181231-nl2code-hearthstone-fef2c5b/att1 --output __LOGDIR__/infer-test-step01900-bs1.jsonl --step 1900 --section test --beam-size 1

WARNING <class 'seq2struct.models.enc_dec.EncDecModel.Preproc'>: superfluous {'name': 'EncDec'}
WARNING <class 'seq2struct.models.enc_dec.EncDecModel'>: superfluous {'decoder_preproc': {'grammar': {'name': 'python'}, 'save_path': 'data/hearthstone/nl2code/', 'max_count': 5000, 'min_freq': 3}, 'encoder_preproc': {'max_count': 5000, 'save_path': 'data/hearthstone/nl2code/', 'min_freq': 3}}
Loading model from logdirs/20181231-nl2code-hearthstone-fef2c5b/att1/model_checkpoint-00001900
100%|███████████████████████████████████████████| 66/66 [00:21<00:00,  3.23it/s]


In [37]:
_, metrics = evaluation.compute_metrics(
    'configs/hearthstone/nl2code.jsonnet', '', 'test',
    'logdirs/20181231-nl2code-hearthstone-fef2c5b/att1/infer-test-step01900-bs1.jsonl')
metrics

OrderedDict([('exact match', 0.12121212121212122),
             ('sentence BLEU', 0.7242459632152278),
             ('corpus BLEU', 0.7335344583509769)])

## Sentence BLEU

In [39]:
flat_df.loc[flat_df['sentence BLEU'].idxmax()]

exp_name              att0
step                  2400
corpus BLEU       0.777414
exact match      0.0909091
sentence BLEU     0.780721
Name: 24, dtype: object

In [40]:
!CUDA_VISIBLE_DEVICES= python infer.py --config configs/hearthstone/nl2code.jsonnet --logdir logdirs/20181231-nl2code-hearthstone-fef2c5b/att0 --output __LOGDIR__/infer-test-step02400-bs1.jsonl --step 2400 --section test --beam-size 1

WARNING <class 'seq2struct.models.enc_dec.EncDecModel.Preproc'>: superfluous {'name': 'EncDec'}
WARNING <class 'seq2struct.models.enc_dec.EncDecModel'>: superfluous {'encoder_preproc': {'min_freq': 3, 'save_path': 'data/hearthstone/nl2code/', 'max_count': 5000}, 'decoder_preproc': {'min_freq': 3, 'grammar': {'name': 'python'}, 'save_path': 'data/hearthstone/nl2code/', 'max_count': 5000}}
Loading model from logdirs/20181231-nl2code-hearthstone-fef2c5b/att0/model_checkpoint-00002400
100%|███████████████████████████████████████████| 66/66 [00:19<00:00,  3.16it/s]


In [41]:
_, metrics = evaluation.compute_metrics(
    'configs/hearthstone/nl2code.jsonnet', '', 'test',\
    'logdirs/20181231-nl2code-hearthstone-fef2c5b/att0/infer-test-step02400-bs1.jsonl')
metrics

OrderedDict([('exact match', 0.19696969696969696),
             ('sentence BLEU', 0.7791194850374037),
             ('corpus BLEU', 0.7638435491009413)])

## Exact match

In [42]:
flat_df.loc[flat_df['exact match'].idxmax()]

exp_name             att1
step                 2300
corpus BLEU      0.762964
exact match       0.19697
sentence BLEU    0.766753
Name: 37, dtype: object

In [43]:
!CUDA_VISIBLE_DEVICES= python infer.py --config configs/hearthstone/nl2code.jsonnet --logdir logdirs/20181231-nl2code-hearthstone-fef2c5b/att1 --output __LOGDIR__/infer-test-step02300-bs1.jsonl --step 2300 --section test --beam-size 1

WARNING <class 'seq2struct.models.enc_dec.EncDecModel.Preproc'>: superfluous {'name': 'EncDec'}
WARNING <class 'seq2struct.models.enc_dec.EncDecModel'>: superfluous {'encoder_preproc': {'max_count': 5000, 'min_freq': 3, 'save_path': 'data/hearthstone/nl2code/'}, 'decoder_preproc': {'max_count': 5000, 'min_freq': 3, 'save_path': 'data/hearthstone/nl2code/', 'grammar': {'name': 'python'}}}
Loading model from logdirs/20181231-nl2code-hearthstone-fef2c5b/att1/model_checkpoint-00002300
100%|███████████████████████████████████████████| 66/66 [00:19<00:00,  3.89it/s]


In [44]:
_, metrics = evaluation.compute_metrics(
    'configs/hearthstone/nl2code.jsonnet', '', 'test',
    'logdirs/20181231-nl2code-hearthstone-fef2c5b/att1/infer-test-step02300-bs1.jsonl')
metrics

OrderedDict([('exact match', 0.15151515151515152),
             ('sentence BLEU', 0.7430320121685502),
             ('corpus BLEU', 0.7229233074922048)])

# Measure idiom popularity

In [53]:
!CUDA_VISIBLE_DEVICES= python infer.py \
    --config configs/hearthstone-idioms/nl2code-0201-allmatches-anysplit-multimean.jsonnet \
    --logdir logdirs/20190201-hs-allmatches-anysplit-multimean/filt-contains-hole_st-cov-examples_nt-80 \
    --config-args "{filt: 'contains-hole', st: 'cov-examples', nt: 80}" \
    --output-history \
    --output __LOGDIR__/infer-test-step02200-bs1-with-history.jsonl --step 2200 --section test --beam-size 1

WARNING <class 'seq2struct.models.enc_dec.EncDecModel.Preproc'>: superfluous {'name': 'EncDec'}
WARNING <class 'seq2struct.models.enc_dec.EncDecModel'>: superfluous {'decoder_preproc': {'grammar': {'name': 'idiom_ast', 'base_grammar': {'name': 'python'}, 'all_sections_rewritten': True, 'template_file': 'data/hearthstone-idioms-20190201/all-matches-trees-anysplit/filt-contains-hole_st-cov-examples_nt-80/templates.json'}, 'max_count': 5000, 'min_freq': 3, 'use_seq_elem_rules': True, 'save_path': 'data/hearthstone-idioms-20190201/all-matches-trees-anysplit/filt-contains-hole_st-cov-examples_nt-80/nl2code/'}, 'encoder_preproc': {'save_path': 'data/hearthstone-idioms-20190201/all-matches-trees-anysplit/filt-contains-hole_st-cov-examples_nt-80/nl2code/', 'max_count': 5000, 'min_freq': 3}}
Loading model from logdirs/20190201-hs-allmatches-anysplit-multimean/filt-contains-hole_st-cov-examples_nt-80/model_checkpoint-00002200
100%|███████████████████████████████████████████| 66/66 [00:19<00:00, 

In [55]:
inferred = [json.loads(line) for line in open('logdirs/20190201-hs-allmatches-anysplit-multimean/filt-contains-hole_st-cov-examples_nt-80/infer-test-step02200-bs1-with-history.jsonl')]

In [63]:
all_rules = json.load(
    open('data/hearthstone-idioms-20190201/all-matches-trees-anysplit/filt-contains-hole_st-cov-examples_nt-80/nl2code/grammar_rules.json')
)['all_rules']

In [72]:
all_template_counts = []
for example in inferred:
    template_counts = collections.Counter()
    for choice in example['beams'][0]['choice_history']:
        if isinstance(choice, int):
            rule = all_rules[choice]
            if isinstance(rule[1], str) and re.match('Template\d+', rule[1]):
                template_counts[tuple(rule)] += 1
    all_template_counts.append(template_counts)
            

In [107]:
print(inferred[0]['beams'][0]['inferred_code'])

class Archmage(MinionCard):

    def __init__(self):
        super().__init__('Archmage', 6, CHARACTER_CLASS.ALL, CARD_RARITY.COMMON
            )

    def create_minion(self, player):
        return Minion(4, 7, spell_damage=1)



In [109]:
[all_rules[choice] if isinstance(choice, int) else choice for choice in inferred[0]['beams'][0]['choice_history']]

[['mod', 'Template4'],
 'Archmage',
 '<EOS>',
 ['expr*', 1],
 ['expr_seq_elem', 'Name'],
 ['Name', ['str']],
 'Minion',
 'Card',
 '<EOS>',
 ['stmt*', 2],
 ['stmt_seq_elem', 'Template13'],
 '__init__',
 '<EOS>',
 ['arguments_plus_templates', 'Template47'],
 ['arg_plus_templates*', 1],
 ['arg_plus_templates_seq_elem', 'arg'],
 ['arg', ['str', False]],
 'self',
 '<EOS>',
 ['stmt*', 1],
 ['stmt_seq_elem', 'Template24'],
 ['Call', [True, True, False]],
 ['expr*', 4],
 ['expr_seq_elem', 'Str'],
 ['Str', ['str']],
 'Archmage',
 '<EOS>',
 ['expr_seq_elem', 'Num'],
 ['Num', ['int']],
 '6',
 '<EOS>',
 ['expr_seq_elem', 'Attribute'],
 ['Attribute', [True, 'str']],
 ['expr', 'Name'],
 ['Name', ['str']],
 'CHARACTER_CLASS',
 '<EOS>',
 'ALL',
 '<EOS>',
 ['expr_seq_elem', 'Attribute'],
 ['Attribute', [True, 'str']],
 ['expr', 'Name'],
 ['Name', ['str']],
 'CARD_RARITY',
 '<EOS>',
 'COMMON',
 '<EOS>',
 ['stmt_seq_elem', 'FunctionDef'],
 ['FunctionDef', ['str', True, True, False, False]],
 'create_mini

## Most frequently selected idioms

In [86]:
all_templates = {
    'Template{}'.format(template['id']): template for template in 
    json.load(open('data/hearthstone-idioms-20190201/all-matches-trees-anysplit/filt-contains-hole_st-cov-examples_nt-80/templates.json'))
}

In [112]:
most_frequent = sum(all_template_counts, collections.Counter())
most_frequent

Counter({('arguments_plus_templates', 'Template33'): 56,
         ('arguments_plus_templates', 'Template36'): 9,
         ('arguments_plus_templates', 'Template46'): 44,
         ('arguments_plus_templates', 'Template47'): 31,
         ('expr', 'Template22'): 2,
         ('expr', 'Template25'): 1,
         ('expr', 'Template39'): 1,
         ('expr', 'Template41'): 1,
         ('expr', 'Template59'): 3,
         ('expr', 'Template68'): 8,
         ('expr', 'Template74'): 1,
         ('expr_seq_elem', 'Template26'): 1,
         ('expr_seq_elem', 'Template42'): 1,
         ('expr_seq_elem', 'Template45'): 1,
         ('expr_seq_elem', 'Template54'): 3,
         ('expr_seq_elem', 'Template55'): 2,
         ('expr_seq_elem', 'Template68'): 19,
         ('expr_seq_elem', 'Template74'): 2,
         ('keyword_plus_templates_seq_elem', 'Template10'): 1,
         ('keyword_plus_templates_seq_elem', 'Template34'): 6,
         ('keyword_plus_templates_seq_elem', 'Template40'): 3,
         ('keywo

In [93]:
import pprint
for (lhs, rhs), count in most_frequent:
    print(rhs)
    pprint.pprint(all_templates[rhs])
    print()

Template4
{'holes': [{'id': 0, 'type': 'AddChild'},
           {'id': 1, 'type': 'AddChild'},
           {'id': 2, 'type': 'ReplaceSelf'},
           {'id': 3, 'type': 'AddChild'},
           {'id': 4, 'type': 'AddChild'},
           {'id': 5, 'type': 'ReplaceSelf'},
           {'id': 6, 'type': 'ReplaceSelf'},
           {'id': 7, 'type': 'ReplaceSelf'}],
 'id': 4,
 'idiom': ['Module',
           [],
           None,
           [['ClassDef',
             [],
             None,
             [['ClassDef-name', [], 0, []],
              ['ClassDef-bases',
               [],
               None,
               [['Name', [], 1, []],
                ['ClassDef-bases', [], None, [['End', [], None, []]]]]],
              ['ClassDef-keywords', [], None, [['End', [], None, []]]],
              ['ClassDef-body',
               [],
               None,
               [['FunctionDef',
                 [],
                 None,
                 [['FunctionDef-name', [], None, [['__init__', [], Non

## Idioms that appear in the greatest number of instances

In [97]:
most_common = collections.Counter(itertools.chain.from_iterable(c.keys() for c in all_template_counts)).most_common(5)
most_common

[(('mod', 'Template4'), 62),
 (('stmt_seq_elem', 'Template24'), 56),
 (('arguments_plus_templates', 'Template33'), 55),
 (('stmt_seq_elem', 'Template13'), 45),
 (('arguments_plus_templates', 'Template46'), 43)]

In [101]:
already_printed = {rhs for (_, rhs), _ in most_frequent}
for (lhs, rhs), count in most_common:
    if rhs in already_printed:
        continue
    print(rhs)
    pprint.pprint(all_templates[rhs])
    print()

Template46
{'holes': [{'id': 0, 'type': 'AddChild'}],
 'id': 46,
 'idiom': ['arguments',
           [],
           None,
           [['arguments-args',
             [],
             None,
             [['arg',
               [],
               None,
               [['arg-arg', [], 0, []],
                ['arg-annotation', [], None, [['Null', [], None, []]]]]],
              ['arguments-args', [], None, [['End', [], None, []]]]]],
            ['arguments-vararg', [], None, [['Null', [], None, []]]],
            ['arguments-kwonlyargs', [], None, [['End', [], None, []]]],
            ['arguments-kw_defaults', [], None, [['End', [], None, []]]],
            ['arguments-kwarg', [], None, [['Null', [], None, []]]],
            ['arguments-defaults', [], None, [['End', [], None, []]]]]]}



In [102]:
len(inferred)

66